### Santander Customer Satisfaction
https://www.kaggle.com/c/santander-customer-satisfaction/data

**Filter Methods**:
1. constant
2. quasi-constant
3. duplicated

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

In [2]:
# load the Santander customer satisfaction dataset from Kaggle
data = pd.read_csv('D:\marlabs\Datasets\santander.csv')
data.shape

(76020, 371)

In [4]:
data.head(3)

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,0


In [5]:
# Target variable will be "TARGET"
x_train, x_test, y_train, y_test = train_test_split(data.drop(labels = ['TARGET'], axis = 1), data['TARGET'], test_size = 0.3, random_state = 123)

#### Remove Constant Features

In [6]:
# Remove constant features (means -> standard deviation equals to 0)
constant_features = [feat for feat in x_train.columns if x_train[feat].std() == 0]

x_train.drop(labels=constant_features, axis=1, inplace=True)
x_test.drop(labels=constant_features, axis=1, inplace=True)

x_train.shape,x_test.shape

((53214, 325), (22806, 325))

#### Remove Quasi Constant Features

In [7]:
sel = VarianceThreshold(threshold=0.02)  # 0.1 indicates 99% of observations approximately

sel.fit(x_train)  # fit finds the features with low variance

sum(sel.get_support()) # how many not quasi-constant?

258

In [8]:
# Only keep these 258 features
features_to_keep = x_train.columns[sel.get_support()]

In [9]:
features_to_keep

Index(['ID', 'var3', 'var15', 'imp_ent_var16_ult1', 'imp_op_var39_comer_ult1',
       'imp_op_var39_comer_ult3', 'imp_op_var40_comer_ult1',
       'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1',
       'imp_op_var40_efect_ult3',
       ...
       'saldo_medio_var29_ult3', 'saldo_medio_var33_hace2',
       'saldo_medio_var33_hace3', 'saldo_medio_var33_ult1',
       'saldo_medio_var33_ult3', 'saldo_medio_var44_hace2',
       'saldo_medio_var44_hace3', 'saldo_medio_var44_ult1',
       'saldo_medio_var44_ult3', 'var38'],
      dtype='object', length=258)

In [10]:
# we can then remove the features like this
x_train = sel.transform(x_train)
x_test = sel.transform(x_test)

x_train.shape, x_test.shape

((53214, 258), (22806, 258))

In [11]:
# sklearn transformations lead to numpy arrays
# here I transform the arrays back to dataframes
# please be mindful of getting the columns assigned
# correctly

x_train= pd.DataFrame(x_train)
x_train.columns = features_to_keep

x_test= pd.DataFrame(x_test)
x_test.columns = features_to_keep

In [12]:
# Check the duplicate feature
# Use two for loops to achieve this
duplicate_feat = []
for i in range(0, len(x_train.columns)):
    col_1 = x_train.columns[i]
    
    for col_2 in x_train.columns[i + 1:]:
        if x_train[col_1].equals(x_train[col_2]):
            duplicate_feat.append(col_2)

#### Remove Highly Correlated Features

In [13]:
# build a dataframe with the correlation between features
# remember that the absolute value of the correlation
# coefficient is important and not the sign

corrmat = x_train.corr()
corrmat = corrmat.abs().unstack() # absolute value of corr coef
corrmat = corrmat.sort_values(ascending=False)
corrmat = corrmat[(corrmat <1)& (corrmat > 0.8)]
corrmat = pd.DataFrame(corrmat).reset_index()
corrmat.columns = ['feature1', 'feature2', 'corr']
corrmat.head()

,feature1,feature2,corr
0,delta_num_aport_var33_1y3,delta_imp_aport_var33_1y3,1.0
1,delta_imp_aport_var33_1y3,delta_num_aport_var33_1y3,1.0
2,delta_num_aport_var17_1y3,delta_imp_aport_var17_1y3,1.0
3,delta_imp_aport_var17_1y3,delta_num_aport_var17_1y3,1.0
4,delta_imp_compra_var44_1y3,delta_num_compra_var44_1y3,1.0


In [17]:
corrmat.feature1.unique().shape

(198,)

.unstack() is a reshaping method. https://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/

In [14]:
# find groups of correlated features

grouped_feature_ls = []
correlated_groups = []

for feature in corrmat.feature1.unique():
    if feature not in grouped_feature_ls:

        # find all features correlated to a single feature
        correlated_block = corrmat[corrmat.feature1 == feature]
        grouped_feature_ls = grouped_feature_ls + list(
            correlated_block.feature2.unique()) + [feature]

        # append the block of features to the list
        correlated_groups.append(correlated_block)

print('found {} correlated groups'.format(len(correlated_groups)))
print('out of {} total features'.format(x_train.shape[1]))

found 63 correlated groups
out of 258 total features


In [22]:
correlated_groups

[                    feature1                   feature2  corr
 0  delta_num_aport_var33_1y3  delta_imp_aport_var33_1y3   1.0,
                     feature1                   feature2  corr
 2  delta_num_aport_var17_1y3  delta_imp_aport_var17_1y3   1.0,
                      feature1                    feature2  corr
 4  delta_imp_compra_var44_1y3  delta_num_compra_var44_1y3   1.0,
                       feature1                   feature2     corr
 6    delta_num_aport_var13_1y3  delta_imp_aport_var13_1y3  1.00000
 555  delta_num_aport_var13_1y3       num_aport_var13_ult1  0.85526,
                 feature1                   feature2      corr
 8   imp_amort_var18_ult1  delta_imp_amort_var18_1y3  1.000000
 83  imp_amort_var18_ult1                 saldo_var1  0.987674,
        feature1                   feature2      corr
 11  saldo_var18  delta_imp_amort_var18_1y3  1.000000
 84  saldo_var18                 saldo_var1  0.987674,
         feature1                 feature2      corr
 12 

In [23]:
# now we can visualise each group. We see that some groups contain
# only 2 correlated features, some other groups present several features 
# that are correlated among themselves.

for group in correlated_groups:
    print(group)
    print()

                    feature1                   feature2  corr
0  delta_num_aport_var33_1y3  delta_imp_aport_var33_1y3   1.0

                    feature1                   feature2  corr
2  delta_num_aport_var17_1y3  delta_imp_aport_var17_1y3   1.0

                     feature1                    feature2  corr
4  delta_imp_compra_var44_1y3  delta_num_compra_var44_1y3   1.0

                      feature1                   feature2     corr
6    delta_num_aport_var13_1y3  delta_imp_aport_var13_1y3  1.00000
555  delta_num_aport_var13_1y3       num_aport_var13_ult1  0.85526

                feature1                   feature2      corr
8   imp_amort_var18_ult1  delta_imp_amort_var18_1y3  1.000000
83  imp_amort_var18_ult1                 saldo_var1  0.987674

       feature1                   feature2      corr
11  saldo_var18  delta_imp_amort_var18_1y3  1.000000
84  saldo_var18                 saldo_var1  0.987674

        feature1                 feature2      corr
12   saldo_var33   s

In [32]:
# we can now investigate further features within one group.
# let's for example select group 9

group = correlated_groups[6]
group

,feature1,feature2,corr
12,saldo_var33,saldo_medio_var33_ult1,0.999911
124,saldo_var33,saldo_medio_var33_ult3,0.983046
621,saldo_var33,saldo_medio_var33_hace2,0.833282


In [26]:
group

,feature1,feature2,corr
12,saldo_var33,saldo_medio_var33_ult1,0.999911
124,saldo_var33,saldo_medio_var33_ult3,0.983046
621,saldo_var33,saldo_medio_var33_hace2,0.833282


In [34]:
# we could select the features with less missing data
# like this:

for feature in list(group.feature2.unique())+['num_var13_corto']:
    print(x_train[feature].isnull().sum())

0
0
0
0


There's no missing values here. <br/>
Alternatively, we could build a machine learning algorithm using all the features from the above list, and select the more predictive one.

In [35]:
from sklearn.ensemble import RandomForestClassifier

features = list(group.feature2.unique())+['num_var13_corto']
rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
rf.fit(x_train[features].fillna(0), y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=39, verbose=0, warm_start=False)

In [36]:
features

['saldo_medio_var33_ult1',
 'saldo_medio_var33_ult3',
 'saldo_medio_var33_hace2',
 'num_var13_corto']

In [37]:
# we get the feature importance attributed by the 
# random forest model (more on this in coming lectures)

importance = pd.concat(
    [pd.Series(features),
     pd.Series(rf.feature_importances_)], axis=1)

importance.columns = ['feature', 'importance']
importance.sort_values(by='importance', ascending=False)

,feature,importance
3,num_var13_corto,0.978335
0,saldo_medio_var33_ult1,0.009707
1,saldo_medio_var33_ult3,0.009084
2,saldo_medio_var33_hace2,0.002873


In this case, feature num_var13_0 shows the higher importance according to random forests. Then, I would select num_var13_0, and remove all the remaining features from this group from the dataset.